In [ ]:
# ----------------- Import Libraries -----------------
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, precision_score, recall_score, classification_report

import seaborn as sns
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
# ----------------- ResNet50_Model Definition -----------------
def ResNet50_Model():
    input_shape = (224, 224, 3)
    resnet50 = tf.keras.applications.ResNet50(include_top=False, input_shape=input_shape, weights='imagenet')

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(units=1024, activation='relu')(x)
    x = layers.Dense(units=512, activation='relu')(x)
    x = layers.Dense(units=256, activation='relu')(x)
    x = layers.Dense(units=128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Dense(units=4, activation='softmax')(x)

    model = models.Model(inputs=resnet50.input, outputs=x, name='ResNet50_Model')
    return model



In [4]:
# ----------------- Compile and Train -----------------
def compile_and_train(model, train_data, test_data, epochs=10, learning_rate=0.001):
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    history = model.fit(
        train_data,
        validation_data=test_data,
        epochs=epochs,
        callbacks=[early_stopping],
        verbose=1
    )
    return history

In [ ]:
# ----------------- Evaluate model performance -----------------
def evaluate_global_model(model, test_loader, client_idx, output_dir="Reset50_model_results"):
    os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists
    output_file = os.path.join(output_dir, f"Model_Results_Client_{client_idx + 1}.txt")
    
    all_preds = []
    all_labels = []
    total_loss = 0

    for features, labels in test_loader:
        features_tf = tf.convert_to_tensor(features, dtype=tf.float32)

        # Convert one-hot encoded labels to class indices
        if len(labels.shape) > 1 and labels.shape[-1] > 1:  # Likely one-hot
            labels_tf = tf.argmax(labels, axis=1, output_type=tf.int32)
        else:
            labels_tf = tf.convert_to_tensor(labels, dtype=tf.int32)

        # Debugging: Print shapes
        print(f"Features shape: {features_tf.shape}, Labels shape: {labels_tf.shape}")

        # Get predictions
        predictions = model(features_tf, training=False)
        predicted = tf.argmax(predictions, axis=1, output_type=tf.int32)

        # Accumulate predictions and labels
        all_preds.extend(predicted.numpy())
        all_labels.extend(labels_tf.numpy())


        # Compute loss
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels_tf, logits=predictions)
        total_loss += tf.reduce_mean(loss).numpy()

    # Compute metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    conf_matrix = confusion_matrix(all_labels, all_preds)

    # Classification report
    class_names = ["glioma", "meningioma", "notumor", "pituitary"] 
    classification_report_str = classification_report(all_labels, all_preds, target_names=class_names)

    # Append metrics to a file
    with open(output_file, "w") as file:
        file.write(f"Evaluation Metrics for Client {client_idx + 1}:\n")
        file.write(f"Accuracy: {accuracy * 100:.2f}%\n")
        file.write(f"Loss: {total_loss / len(test_loader):.4f}\n")
        file.write(f"Precision: {precision:.4f}\n")
        file.write(f"Recall: {recall:.4f}\n")
        file.write(f"F1 Score: {f1:.4f}\n\n")
        file.write("Classification Report:\n")
        file.write(classification_report_str)
        file.write("\nConfusion Matrix:\n")
        file.write("\n".join(["\t".join(map(str, row)) for row in conf_matrix]))
        file.write("\n" + "-" * 50 + "\n")

    # Print metrics
    print("\nClassification Report:\n", classification_report_str)
    print(f"\nAccuracy: {accuracy * 100:.2f}%")
    print(f"Loss: {total_loss / len(test_loader):.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    # Plot and save heatmap
    plt.figure(figsize=(10, 7))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title(f'Confusion Matrix (Client {client_idx + 1})')
    plt.savefig(os.path.join(output_dir, f"confusion_matrix_Client_{client_idx + 1}.png"))
    plt.close()


### Replace path with the respective NON-IID dataset paths

In [ ]:
# Directories for each client

split_dirs = [
    "../NON_IID/Model_1",
    "../NON_IID/Model_2",
    "../NON_IID/Model_3",
    "../NON_IID/Model_4"
]

### Load the NON-IID dataset

In [7]:
# Updated image transformations: Normalize first, then permute
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),          # Convert image to tensor format (C, H, W)
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
    transforms.Lambda(lambda x: x.permute(1, 2, 0))  # Permute to (H, W, C)
])

def get_tumor_dataloaders(split_dirs, batch_size=25, shuffle=True):
    """
    Returns data loaders for all clients for both training and testing sets.
    """
    tumor_iid_train_dls = []
    tumor_iid_test_dls = []

    for client_idx, client_dir in enumerate(split_dirs):
        # Get the directory for the current client
        train_dir = os.path.join(client_dir, 'train')
        test_dir = os.path.join(client_dir, 'test')

        # Check if the directories exist
        if not os.path.exists(train_dir) or not os.path.exists(test_dir):
            print(f"Directory not found for client {client_idx + 1}:")
            print(f"Train dir: {train_dir}")
            print(f"Test dir: {test_dir}")
            continue

        # Load training data for the current client
        train_dataset = datasets.ImageFolder(train_dir, transform=transform)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)

        # Load testing data for the current client
        test_dataset = datasets.ImageFolder(test_dir, transform=transform)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        # Append the dataloaders for the current client to the list
        tumor_iid_train_dls.append(train_loader)
        tumor_iid_test_dls.append(test_loader)

    return tumor_iid_train_dls, tumor_iid_test_dls

# Get the training and testing data loaders
Tumor_iid_train_dls, Tumor_iid_test_dls = get_tumor_dataloaders(split_dirs, batch_size=25)

# Checking the sizes of the images in the data loaders to verify the shape
for batch_idx, (images, labels) in enumerate(Tumor_iid_train_dls[0]):  # Checking for client 1
    print(f"Batch {batch_idx} image sizes: {images.size()}")  # Should print torch.Size([25, 224, 224, 3])
    break  # Check only the first batch


Batch 0 image sizes: torch.Size([25, 224, 224, 3])


### Converts a PyTorch DataLoader to a TensorFlow dataset.

In [ ]:
from tensorflow.keras.utils import to_categorical

def pytorch_to_tf_dataset(pytorch_loader, num_classes):
    images, labels = [], []
    for batch_images, batch_labels in pytorch_loader:
        # Convert PyTorch tensors to NumPy arrays and permute to (Batch, Height, Width, Channels)
        images.append(batch_images.permute(0, 2, 1, 3).numpy())
        # One-hot encode labels
        labels.append(to_categorical(batch_labels.numpy(), num_classes=num_classes))
    
    # Concatenate all batches
    images = np.concatenate(images, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    # Create TensorFlow dataset
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    return dataset.batch(pytorch_loader.batch_size)




### Main method

In [ ]:
if __name__ == "__main__":
    # Iterate over all clients' datasets, here 4 clients, 4 datasets
    for client_idx in range(len(split_dirs)):
        print(f"\nTraining and testing on dataset: Model_{client_idx + 1}\n")
        
        # Load data for the current client
        train_loader = Tumor_iid_train_dls[client_idx]
        test_loader = Tumor_iid_test_dls[client_idx]

        # Number of classes in your dataset
        num_classes = 4

        # Convert PyTorch DataLoaders to TensorFlow datasets
        tf_train_dataset = pytorch_to_tf_dataset(train_loader, num_classes)
        tf_test_dataset = pytorch_to_tf_dataset(test_loader, num_classes)

        # Build Model
        model = ResNet50_Model()

        # Use these datasets for training
        history = compile_and_train(model, tf_train_dataset, tf_test_dataset, epochs=10, learning_rate=0.0001)

        # Evaluate and Save Results
        evaluate_global_model(model, tf_test_dataset, client_idx, output_dir="Reset50_model_results")

        print(f"Training and evaluation completed for Model_{client_idx + 1}\n")
